In [19]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

In [2]:
(train_ds, train_labels), (test_ds, test_labels) = tfds.load("tf_flowers", split=["train[:70%]", "train[:30%]"],
                                                             batch_size=-1, as_supervised=True)

In [3]:
print(train_ds.shape)
print(train_ds[0].shape)

(2569, 442, 1024, 3)
(442, 1024, 3)


In [4]:
#resizing images
train_ds = tf.image.resize(train_ds, (150,150))
test_ds = tf.image.resize(test_ds, (150,150))

In [5]:
train_labels

<tf.Tensor: shape=(2569,), dtype=int64, numpy=array([2, 3, 3, ..., 0, 2, 0])>

In [6]:
#Transform labels to correct format
train_labels = to_categorical(train_labels, num_classes=5)
test_labels = to_categorical(test_labels, num_classes=5)

In [7]:
train_labels[0]

array([0., 0., 1., 0., 0.], dtype=float32)

In [8]:
#use pretrained VGG16 image Classification model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [9]:
#loading VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=train_ds[0].shape)

In [10]:
# will not train base model
base_model.trainable = False
#preprocessing input
train_ds = preprocess_input(train_ds)
test_ds = preprocess_input(test_ds)

In [11]:
# model details
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [12]:
#add our layers on top of the model
from tensorflow.keras import layers, models
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')

ml= models.Sequential([base_model, flatten_layer, dense_layer_1, dense_layer_2, prediction_layer])

In [13]:
from tensorflow.keras.callbacks import EarlyStopping
ml.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True)

In [15]:
ml.fit(train_ds, train_labels, epochs=10, validation_split=0.2, batch_size=32, callbacks=[es])

Epoch 1/10
65/65 [==============================] - 22s 126ms/step - loss: 1.9612 - accuracy: 0.4438 - val_loss: 1.1778 - val_accuracy: 0.5759
Epoch 2/10
65/65 [==============================] - 5s 82ms/step - loss: 0.8988 - accuracy: 0.6856 - val_loss: 1.0345 - val_accuracy: 0.6615
Epoch 3/10
65/65 [==============================] - 5s 81ms/step - loss: 0.5765 - accuracy: 0.7893 - val_loss: 0.9024 - val_accuracy: 0.7023
Epoch 4/10
65/65 [==============================] - 5s 81ms/step - loss: 0.4293 - accuracy: 0.8472 - val_loss: 0.8941 - val_accuracy: 0.7160
Epoch 5/10
65/65 [==============================] - 6s 86ms/step - loss: 0.2802 - accuracy: 0.8949 - val_loss: 0.9657 - val_accuracy: 0.7101
Epoch 6/10
65/65 [==============================] - 5s 81ms/step - loss: 0.2034 - accuracy: 0.9260 - val_loss: 1.0207 - val_accuracy: 0.7043
Epoch 7/10
65/65 [==============================] - 6s 86ms/step - loss: 0.1560 - accuracy: 0.9392 - val_loss: 1.0422 - val_accuracy: 0.6887
Epoch 8/10


In [16]:
ml.evaluate(test_ds, test_labels)

35/35 [==============================] - 3s 99ms/step - loss: 0.2966 - accuracy: 0.9083


[0.29659321904182434, 0.908265233039856]

In [17]:
#model build from scratch
from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

# layers
ml = Sequential()
ml.add(Rescaling(1./255, input_shape=(150,150,3)))
ml.add(layers.Conv2D(16,kernel_size=8, activation='relu'))
ml.add(layers.MaxPooling2D(3))
ml.add(layers.Conv2D(32,kernel_size=8, activation='relu'))
ml.add(layers.MaxPooling2D(2))
ml.add(layers.Conv2D(32,kernel_size=8, activation='relu'))
ml.add(layers.MaxPooling2D(2))

# Flatten
ml.add(layers.Flatten())
ml.add(layers.Dense(50, activation='relu'))
ml.add(layers.Dense(20, activation='relu'))
ml.add(layers.Dense(5, activation='softmax'))

# Compile
ml.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'],)

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True)
ml.fit(train_ds,train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])

Epoch 1/50
65/65 [==============================] - 6s 30ms/step - loss: 1.6007 - accuracy: 0.2467 - val_loss: 1.5680 - val_accuracy: 0.3152
Epoch 2/50
65/65 [==============================] - 1s 22ms/step - loss: 1.4468 - accuracy: 0.3591 - val_loss: 1.2718 - val_accuracy: 0.4533
Epoch 3/50
65/65 [==============================] - 1s 19ms/step - loss: 1.2717 - accuracy: 0.4448 - val_loss: 1.2500 - val_accuracy: 0.4222
Epoch 4/50
65/65 [==============================] - 1s 21ms/step - loss: 1.1894 - accuracy: 0.4852 - val_loss: 1.1649 - val_accuracy: 0.5156
Epoch 5/50
65/65 [==============================] - 1s 22ms/step - loss: 1.1489 - accuracy: 0.5241 - val_loss: 1.1630 - val_accuracy: 0.5039
Epoch 6/50
65/65 [==============================] - 1s 19ms/step - loss: 1.0982 - accuracy: 0.5401 - val_loss: 1.1351 - val_accuracy: 0.5253
Epoch 7/50
65/65 [==============================] - 1s 19ms/step - loss: 1.0622 - accuracy: 0.5650 - val_loss: 1.1370 - val_accuracy: 0.5097
Epoch 8/50
65

In [18]:
ml.evaluate(test_ds, test_labels)

35/35 [==============================] - 1s 16ms/step - loss: 0.7066 - accuracy: 0.7275


[0.7066217064857483, 0.7275204062461853]